In [1]:
import pandas as pd
import csv
from collections import defaultdict
import sqlite3

class Caption:
    def __init__(self, items):
        self.idx = items[0]
        self.file_id = items[1]
        self.filename = self.file_id.replace(".jpg", ".txt")
        self.caption = items[2].strip().strip(".").strip()
        self.author = items[3]
        self.is_occluded = items[4]
        self.is_error = items[5]

    def __str__(self):
        return "id:{},file_id:{},filename:{},caption:{},user:{},is_occleded:{},is_error:{}".format(
            self.idx, self.file_id, self.filename, 
            self.caption, self.author, self.is_occluded, self.is_error
        )

/home/soumen/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# dicta = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
# with open("caption.csv" , "r") as fp:
#     csvfp = csv.reader(fp)
#     for v in csvfp:
#         cap = Caption(v)
#         dicta[cap.file_id][cap.author][cap.caption] += 1
# for file_id, user_cap_count in dicta.items():
#     for user, caption_count in user_cap_count.items():
#         total = 0
#         for caption, count in caption_count.items():
#             total += count
#             if count > 1:
#                 print("DUPLICATE:", file_id, user, caption, count)
#         if total > 2:
#             print("MORE 2:", file_id, user, caption)
    

# Read form SQLite

In [3]:
# !scp anshul@10.73.24.39:/home/anshul/app/tip_gai.db ./latest_tip_gai.db

In [17]:
conn = sqlite3.connect("latest_tip_gai.db")
curr = conn.cursor()
df = pd.read_sql_query("SELECT * FROM caption", conn)
df['one'] = 1
df.head()


,id,file_id,caption,user,is_error,is_occluded,one
0,1,P00001.jpg,"Two knives, one placed on top of other, in a b...",anshul,0,1,1
1,2,P00001.jpg,A bag with knives.\nTwo shrap knives in a back...,soumen,0,1,1
2,3,P00001.jpg,Security discovered a concealed knife in the p...,soumen,0,1,1
3,5,P00004.jpg,Two knives are hidden inside a backpack.,soumen,0,0,1
4,6,P00005.jpg,Two knives are hidden inside a backpack overla...,soumen,0,0,1


In [16]:
#!pip install language-tool-python
# import language_tool_python
def error_check_and_correct(text, correction=False):
    # using the tool
    my_tool = language_tool_python.LanguageTool('en-US')

    # getting the matches
    my_matches = my_tool.check(text)

    # defining some variables
    myMistakes = []
    if correction:
        myCorrections = []
    startPositions = []
    endPositions = []

    # using the for-loop
    for rules in my_matches:
        if len(rules.replacements) > 0:
            startPositions.append(rules.offset)
            endPositions.append(rules.errorLength + rules.offset)
            myMistakes.append(my_text[rules.offset : rules.errorLength + rules.offset])
            if correction:
                myCorrections.append(rules.replacements[0])
    
    if correction:
        # creating new object
        my_NewText = list(my_text)

        # rewriting the correct passage
        for n in range(len(startPositions)):
            for i in range(len(my_text)):
                my_NewText[startPositions[n]] = myCorrections[n]
                if (i > startPositions[n] and i < endPositions[n]):
                    my_NewText[i] = ""

        my_NewText = "".join(my_NewText)

        # printing the text
        return my_NewText, list(zip(myMistakes, myCorrections))
    else:
        return len(myMistakes) == 0
# error_check_and_correct("He go to shchool", True)

In [6]:
def statistics(conn, log_=False, linefeed="\n"):
    def sprint(*args, end="\n"):
        return args[0] + " ".join(map(str, args[1:])) + end
    
    df = pd.read_sql_query("SELECT * FROM caption", conn)
    tgt = 2500
    s = ""
    s = sprint(s, "## Target Captions:", tgt, end=linefeed)
    s = sprint(s, "## Captions:", df.shape[0], end=linefeed)
    s = sprint(s, "## Captions %:", df.shape[0]/ tgt * 100, end=linefeed)
    error_marked = df['is_error'].sum()
    s = sprint(s, "{}#### QUALITY STAT ####".format(linefeed))
    s = sprint(s, "## Error Sentence marked:", error_marked, end=linefeed)
    s = sprint(s, "## Error Sentence marked %:", round(error_marked / df.shape[0] * 100, 3), end=linefeed)
    df['one'] = 1
    gp = df.groupby(by=['file_id', 'caption'])
    tdf = gp.count()['one']
    s = sprint(s, "{}## Duplicate:".format(linefeed), end=linefeed)
    s = sprint(s, tdf[tdf > 1], end=linefeed)
    s = sprint(s, "{}## Malformated:".format(linefeed), end=linefeed)
    s = sprint(s, df[df['caption'].apply(lambda x: len(x.split(' ')) < 4)][['file_id', 'caption']], end=linefeed)
    s = sprint(s, "{}#### USER STAT ####".format(linefeed))
    s = sprint(s, "## Caption Count/user:", end=linefeed)
    df['user'] = df['user'].map(dict(anshul="user-1",
                                     soumen="user-2",
                                     sriram="user-3",
                                     tejas="user-4",
                                     murugan="user-5",
                                     purus="user-6",
                                    ))
    s = sprint(s, df['user'].value_counts(sort=False), end=linefeed)
    s = sprint(s, "{}## Caption %/user:".format(linefeed), end=linefeed)
    s = sprint(s, df['user'].value_counts(sort=False) / tgt * 100, end=linefeed)
    s = s.replace("\n", linefeed).replace("\r", linefeed)
    
    if log_:
        print(s)
    return s
print(statistics(conn, linefeed="\n"))

## Target Captions: 2500
## Captions: 2013
## Captions %: 80.52

#### QUALITY STAT ####
## Error Sentence marked: 26
## Error Sentence marked %: 1.292

## Duplicate:
file_id     caption                                                            
P00004.jpg  Two knives intersecting each other on left side of bag                 2
P02864.jpg  travel backpack found to have two guns and a knife hidden inside it    2
Name: one, dtype: int64

## Malformated:
        file_id caption
361  P02796.jpg    1745

#### USER STAT ####
## Caption Count/user:
user-1       7
user-2     762
user-3    1124
user-4      15
user-5      74
Name: user, dtype: int64

## Caption %/user:
user-1     0.28
user-2    30.48
user-3    44.96
user-4     0.60
user-5     2.96
Name: user, dtype: float64



In [7]:
df.head()

,id,file_id,caption,user,is_error,is_occluded
0,1,P00001.jpg,"Two knives, one placed on top of other, in a b...",anshul,0,1
1,2,P00001.jpg,A bag with knives.\nTwo shrap knives in a back...,soumen,0,1
2,3,P00001.jpg,Security discovered a concealed knife in the p...,soumen,0,1
3,5,P00004.jpg,Two knives are hidden inside a backpack.,soumen,0,0
4,6,P00005.jpg,Two knives are hidden inside a backpack overla...,soumen,0,0


In [8]:
# User wise caption Count
df['file_id'].unique().shape

(500,)

In [9]:
# !pip install nltk
import nltk
from nltk.corpus import wordnet
# nltk.download('punkt')
# nltk.download('wordnet')
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def paraphrase_text(text):
    paraphrased_text = []
    words = nltk.word_tokenize(text)
    for word in words:
        synonyms = get_synonyms(word)
        if synonyms:
            paraphrased_text.append(synonyms[0])
        else:
            paraphrased_text.append(word)
    return ' '.join(paraphrased_text)

# # Example usage
# original_text = "A trolley bag contains a sharp knife."
# paraphrased_text = paraphrase_text(original_text)

# print("Original text:", original_text)
# print("Paraphrased text:", paraphrased_text)


In [10]:
# !pip install transformers
from transformers import AutoModelWithLMHead, AutoTokenizer

def paraphrase_text(text):
    model_name = "t5-base"  # Pre-trained model name
    model = AutoModelWithLMHead.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Tokenize the input text
    input_ids = tokenizer.encode(text, return_tensors="pt")

    # Generate paraphrased text
    paraphrased_ids = model.generate(input_ids, max_length=50, num_return_sequences=1, early_stopping=True)
    paraphrased_text = tokenizer.decode(paraphrased_ids[0], skip_special_tokens=True)

    return paraphrased_text

# Example usage
# original_text = "a gun, an Wi-Fi router and a sharp object with a water bottle inside a trolly bag."
# paraphrased_text = paraphrase_text(original_text)

# print("Original text:", original_text)
# print("Paraphrased text:", paraphrased_text)


# OPEN-AI ChatGPT

In [11]:
# !pip install -q openai

In [12]:
import openai

# load API key
# with open("openai.apikey", "r") as fp:
openai.api_key_path = "openai.apikey"

In [13]:
def send_message(message):
    payload = {
      "model": "text-davinci-003",
      "messages": [{"role": "user", "content": message}]
    }
    chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=[payload]
    )
    return chat

In [14]:
# send_message("Hi!")


# Read captions for `user!=soumen`

In [25]:
# count files with [1,2,3,4] captions
file_count_series = df.groupby(by='file_id').count()['one']

files_1234 = file_count_series[file_count_series <= 4]
files_1234.shape

(200,)

(199,)

In [33]:
# not my files and less than 5 captions
soumen_files = df[df['user'] == "soumen"].file_id.unique()
dfnotmy = df[~df['file_id'].isin(soumen_files)]
df2 = dfnotmy[dfnotmy['file_id'].isin(files_1234.index)]
df2.shape, df2.file_id.unique().shape, df2.user.unique(), dfnotmy.user.unique()

((513, 7),
 (200,),
 array(['sriram', 'murugan', 'purush'], dtype=object),
 array(['sriram', 'murugan', 'purush'], dtype=object))

In [58]:
# random 200 records
dfs = df2.sample(300, random_state=23)
dfs.shape

(300, 7)

In [61]:
s = ""
ctr = 0
i = 1
print("please paraphrease the lines:")
with open("original.txt", "w") as fp:
    for x in dfs.iloc[:500,:].caption.unique():
        if ctr > 2000:
            print("\nplease paraphrease the lines:")
            ctr = 0
            i = 1
        ctr += len(x)
        txt = x.strip(".").strip().replace(".", " ").replace("..", ".")
#         fp.write(f"{i}. ")
        fp.write(txt)
        fp.write("\n")
        print(f"{i}. {txt}")
        i+=1

please paraphrease the lines:
1. Backpack with chopping knife at the bottom on the bag
2. A bag containing a laptop was discovered with two knives
3. two guns and a knife hidden inside a travel backpack at the bottom
4. A knife in a tray along with other Items
5. the trolley bag concealed a total of three knives and two guns, cleverly positioned at its lower section
6. The luggage holds various tools, and there are five guns hidden in scattered bottles
7. The trolley bag contained two guns cleverly hidden among a mess of wires, making them difficult to spot
8. A trolly bag hides two knives behind the clothes
9. At the bottom of the trolley bag, security personnel detected the presence of two knives and a gan
10. backpack found to have two guns and a knife hidden inside it
11. A gun was discovered in a travel bag, alongside a few bottles, during the inspection
12. A knife, along with a laptop and other electronic items, was found in the trolley bag during the inspection
13. t was observ

In [65]:
# preapre caption replacement dictionary
with open("mapped.txt", "r") as fp:
    lines = fp.readlines()

with open("original.txt", "r") as fp:
    captions = fp.readlines()

caption1_map = defaultdict(lambda: [])
remain = []
for i in range(len(captions)):
    caption1_map[captions[i].strip().strip('.').strip().replace(".", " ")].append(lines[i] if i < len(lines) else "")
    if i >= len(lines):
        remain.append(captions[i])
for x in remain:
    print(x)

In [66]:
import random
not_found = []
def caption_paraphreaser(caption):
    cleaned = caption.strip().strip('.').strip().replace(".", " ")
    alternative_captions = caption1_map[cleaned]
    if len(alternative_captions) == 0:
        print("NOT AVAILABLE: '{}', '{}'".format(caption, cleaned))
        not_found.append(caption)
        return "Two sharp knives are inside a backpack."
    if len(alternative_captions) == 1:
        return alternative_captions[0]
    idx = random.randint(0, len(alternative_captions)-1)
    return alternative_captions[idx]

In [67]:
# replace captions
dfs['caption_2'] = dfs['caption'].apply(caption_paraphreaser)

In [69]:
dfs.to_csv("bulk_upload_soumen_3.csv", index=False)